In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
import tensorflow_hub as hub

from keras_preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
data_dir = '../input/chest-xray-pneumonia/chest_xray'

test_dir = os.path.join(data_dir, 'test/')
train_dir = os.path.join(data_dir, 'train/')
valid_dir = os.path.join(data_dir, 'val/')

model_file = os.path.join('../input/model6', 'model.h5')

In [3]:
test_datagen = ImageDataGenerator(rescale=1./255.)

In [4]:
test_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=False,
    seed=42,
    classes=['NORMAL', 'PNEUMONIA']
)

train_generator = test_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=False,
    seed=42,
    classes=['NORMAL', 'PNEUMONIA']
)

valid_generator = test_datagen.flow_from_directory(
    directory=valid_dir,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=False,
    seed=42,
    classes=['NORMAL', 'PNEUMONIA']
)

Found 624 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [5]:
model = tf.keras.models.load_model(
    model_file,
    custom_objects={
        'KerasLayer': hub.KerasLayer
    }
)

In [6]:
loss, accuracy = model.evaluate(valid_generator)

print('\nLoss: {:,.3f}'.format(loss))
print('Accuracy: {:.3%}'.format(accuracy))

1/1 [==============================] - 0s 3ms/step - loss: 0.6743 - accuracy: 0.5625

Loss: 0.674
Accuracy: 56.250%


In [7]:
loss, accuracy = model.evaluate(train_generator)

print('\nLoss: {:,.3f}'.format(loss))
print('Accuracy: {:.3%}'.format(accuracy))

163/163 [==============================] - 63s 388ms/step - loss: 0.7041 - accuracy: 0.5305

Loss: 0.704
Accuracy: 53.048%


In [8]:
loss, accuracy = model.evaluate(test_generator)

print('\nLoss: {:,.3f}'.format(loss))
print('Accuracy: {:.3%}'.format(accuracy))

20/20 [==============================] - 7s 336ms/step - loss: 0.6872 - accuracy: 0.5625

Loss: 0.687
Accuracy: 56.250%


---
---
---

In [9]:
batch_size = 32
num_of_train_samples = 5216

In [10]:
Y_pred = model.predict_generator(train_generator, num_of_train_samples//batch_size+1, verbose=1)
y_pred = np.argmax(Y_pred, axis=1)

163/164 [============================>.] - 60s 369ms/step


In [11]:
print('Confusion Matrix')
print(confusion_matrix(train_generator.classes, y_pred))

Confusion Matrix
[[ 912  429]
 [2020 1855]]


In [12]:
print('Classification Report')
target_names = ['Normal', 'Pneumonia']
print(classification_report(train_generator.classes, y_pred, target_names=target_names))

Classification Report
              precision    recall  f1-score   support

      Normal       0.31      0.68      0.43      1341
   Pneumonia       0.81      0.48      0.60      3875

    accuracy                           0.53      5216
   macro avg       0.56      0.58      0.51      5216
weighted avg       0.68      0.53      0.56      5216

